# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [63]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel/grip_fixture_low_force/" # SET
dataset_name = "grip_fixture_low_force"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = " "  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-292409.xml.json"

zynq_data_csv = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel/grip_fixture_low_force/grip_fixture_low_force.csv"


unit_convert_path = dataset_path + "/" + dataset_name + "_unitConvert.csv"


ORIGINAL_FREQ = 10000

INTERPOLATE = True


FILTER = True
FILTER_FREQ = 30
FILTER_VELOCITY = True
FILTER_POSITION = True

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 60
DOWNSAMPLE_MOVING_AVERAGE = True

##################################################################################

# EMBEDDED

In [64]:
!. ~/catkin_ws/devel/setup.bash
from downsample import downsample_dataframe
import pandas as pd 
import os

## Unit Convert

In [65]:
# !.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

## Extract Force Sensor Data

In [66]:
import filter
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv

df = pd.read_csv( os.path.join(dataset_path,"sensor","sensor.csv") , header=None)

### FILTERING Force Sensor 

if (FILTER):

    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)
    df = filter.apply_filter_to_fs_df(df, fir_coeffs)

if (DOWNSAMPLE):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)

if (INTERPOLATE):
    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
        print("INTERPOLATE DOWNSAMPLE")
    else:
        freq = ORIGINAL_FREQ
    
    df = interpolate_dataframe_to_sample_rate(df, freq)

df.to_csv(os.path.join(dataset_path,"sensor","sensor.csv"), index=False, header=False)



Saved SENSOR DATA CSV to /home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel/grip_fixture_low_force//sensor/sensor.csv
INTERPOLATE DOWNSAMPLE
[0.00000000e+00 1.65848042e-02 3.31847944e-02 ... 1.31073585e+02
 1.31090185e+02 1.31106785e+02]


## Extract Joint Data

In [67]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

## Filter Joint Data

### Preprocessed Training File Paths

In [68]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


### FIR Filtering

In [69]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)

### Downsample


In [70]:


if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### Interpolate

In [71]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):

    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
    else:
        freq = ORIGINAL_FREQ
    
    df = interpolate_dataframe_to_sample_rate(df, freq)


[0.00000000e+00 1.65848042e-02 3.31847944e-02 ... 1.31073585e+02
 1.31090185e+02 1.31106785e+02]


### SAVE TO CSV

In [72]:
df.to_csv(data_file, index=False, header=False)

# Extract Jacobian

In [73]:
!mkdir {dataset_path}jacobian

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")

!.venv/bin/python3 interpolate_jacobian2.py {data_file} {dataset_path}jacobian/interpolated_all_jacobian.csv dvpsm.rob


FIRST JACOBIAN VALUE
[[ 4.16333634e-17 -2.08166817e-17  7.60207060e-02 -1.38777878e-17
  -6.43216412e-02 -9.07700798e-02]
 [ 3.46944695e-18 -1.04083409e-17  7.15725828e-01  2.60208521e-18
   6.23517481e-02 -4.34198170e-02]
 [-1.04083409e-17  3.46944695e-18 -6.94231511e-01  2.60208521e-18
   5.72388020e-02 -5.47037832e-02]
 [-5.55111512e-17 -9.94057889e-01  0.00000000e+00  7.60207060e-02
  -7.92548036e-01  6.09805467e-01]
 [-1.00000000e+00 -5.55111512e-17  0.00000000e+00  7.15725828e-01
  -3.79114910e-01 -4.89874878e-01]
 [-5.55111512e-17 -1.08852712e-01  0.00000000e+00 -6.94231511e-01
  -4.77639504e-01 -6.23024796e-01]]
Flattened Jacobians (column-major) written to /home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel/grip_fixture_low_force/jacobian/interpolated_all_jacobian.csv


## Interpolate

In [74]:


if (INTERPOLATE == True):

    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
    else:
        freq = ORIGINAL_FREQ
    
    
    jacobian_path = os.path.join(dataset_path,"jacobian" ,"interpolated_all_jacobian.csv")
    df = pd.read_csv( jacobian_path, header=None)
    df = interpolate_dataframe_to_sample_rate(df, freq)
    df.to_csv(jacobian_path, index=False, header=False)


[0.00000000e+00 1.66675292e-02 3.33350584e-02 ... 1.31073450e+02
 1.31090117e+02 1.31106785e+02]


In [75]:
# REMOVING JACOBIAN FILTERING -> MATHEMATICALLY INCORRECT


# df = pd.read_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), header=None)

# if DOWNSAMPLE == True:
#     df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
#     df.to_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), index=False, header=False)

# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE